In [1]:
import numpy as np
import pandas as pd

In [2]:
rag_features=np.load("rag_features.npy")

In [4]:
cnn_features=pd.read_csv("cnn_features.csv")

In [9]:
cnn_features=cnn_features.values

In [10]:
rag_features.shape

(10015, 12100)

In [11]:
cnn_features.shape

(10015, 10817)

In [13]:
total_features=np.hstack((rag_features,cnn_features))

In [14]:
total_features.shape

(10015, 22917)

In [15]:
def get_Y_orig(labels_loc):
    #output_list=list()
    Y_df = pd.read_csv(labels_loc)
    Y_df=Y_df.iloc[:,1:]
#     for index,row in Y_df.iterrows():
#         if row["MEL"]==1:
#             output_list.append(0)
#         elif row["NV"]==1:
#             output_list.append(1)
#         elif row["BCC"]==1:
#             output_list.append(2)
#         elif row["AKIEC"]==1:
#             output_list.append(3)
#         elif row["BKL"]==1:
#             output_list.append(4)
#         elif row["DF"]==1:
#             output_list.append(5)
#         elif row["VASC"]==1:
#             output_list.append(6)
    return Y_df

In [16]:
labels_loc = '/home/sirzechlucifer/Work/CVPR_graphs/ISIC2018_target/ISIC2018_Task3_Training_GroundTruth.csv'

Y_orig = get_Y_orig(labels_loc)

In [17]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=22917, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(7, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split
train_X,val_X,train_Y,val_Y= train_test_split(total_features, Y_orig, test_size=0.2, shuffle= True)

In [ ]:
model.fit(train_X, train_Y, epochs=100, batch_size=64,shuffle=True)

In [ ]:
_, accuracy = model.evaluate(val_X, val_Y)
print('Accuracy: %.2f' % (accuracy*100))